### Imports

In [457]:
import pandas as pd
from sklearn.model_selection import cross_val_score, KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from pathlib import Path


### Load Datasets

In [458]:
data_path = Path.cwd()

# Pfad zur CSV-Datei das Später erstellt wird
train = pd.read_csv(data_path/'../csv/feature_data/train_engineered.csv')
test = pd.read_csv(data_path/'../csv/feature_data/test_engineered.csv')


In [459]:
print(len(train))
print(len(test))

891
418


### Modell:  XGBoost-Klassifikator

In [460]:
import pandas as pd
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from pathlib import Path


data_path = Path.cwd()

# Pfad zur CSV-Datei das Später erstellt wird
train = pd.read_csv(data_path/'../csv/feature_data/train_engineered.csv')
test = pd.read_csv(data_path/'../csv/feature_data/test_engineered.csv')



print(len(train))
print(len(test))

# Angenommen, deine Trainings- und Testdatensätze wurden bereits geladen in `train` und `test`

# Setze 'Survived' im Testdatensatz auf -1, um die Struktur beizubehalten
test['Survived'] = -1

# Entferne die 'Ticket' Spalte aus beiden Datensätzen
# train = train.drop(['Ticket'], axis=1)
# test = test.drop(['Ticket'], axis=1)

# Definiere Features und Label
X_train = train.drop(['PassengerId', 'Survived'], axis=1)
y_train = train['Survived']







# Vorbereitung des Modells
# Hier verwenden wir XGBClassifier direkt, da XGBoost seine eigene Methode zur Handhabung von Feature-Skalierung hat,
# und es ist nicht zwingend notwendig, StandardScaler in der Pipeline zu verwenden.
model = XGBClassifier(n_estimators=100, random_state=7, use_label_encoder=False, eval_metric='logloss')

# Cross-Validation
cv = KFold(n_splits=15, random_state=42, shuffle=True)
scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='accuracy')
print(f'Accuracy scores for the 10 folds: {scores}')
print(f'Average accuracy: {scores.mean()}')

# Trainiere das Modell mit dem gesamten Trainingsdatensatz
model.fit(X_train, y_train)

# Mache Vorhersagen für den Testdatensatz
X_test = test.drop(['PassengerId', 'Survived'], axis=1)
predictions = model.predict(X_test)

# Füge Vorhersagen in den Testdatensatz ein und exportiere in CSV
test['Survived'] = predictions
result = test[['PassengerId', 'Survived']]
result.to_csv(data_path / '../csv/submission_test.csv', index=False)


891
418
Accuracy scores for the 10 folds: [0.88333333 0.85       0.91666667 0.9        0.9        0.83333333
 0.93220339 0.96610169 0.91525424 0.86440678 0.81355932 0.84745763
 0.84745763 0.88135593 0.93220339]
Average accuracy: 0.8855555555555554


Die Auswahl der wichtigsten Features für ein Modell, insbesondere wenn du XGBoost verwendest, kann die Modellleistung erheblich verbessern und die Trainingszeit reduzieren. Es gibt mehrere Ansätze zur Feature-Auswahl, die du in Betracht ziehen kannst. Hier sind einige gängige Methoden, die effektiv mit XGBoost funktionieren:

### 1. **XGBoost's Built-In Feature Importance**

XGBoost bietet eingebaute Funktionen, um die Wichtigkeit der Features zu bewerten. Nach dem Trainieren deines Modells kannst du `model.feature_importances_` verwenden, um zu sehen, welche Features das Modell als am wichtigsten erachtet. Diese Methode ist schnell und einfach, aber sie gibt dir nur einen groben Überblick über die Feature-Wichtigkeit basierend auf der Anzahl der male, die ein Feature für das Splitting verwendet wurde, und der Verbesserung der Performance durch diese Splits.

### 2. **Recursive Feature Elimination (RFE)**

Die Recursive Feature Elimination (RFE) ist ein Verfahren, das ein Modell trainiert und die am wenigsten wichtigen Features schrittweise entfernt. Du kannst RFE mit XGBoost verwenden, indem du es mit dem RFE-Wrapper von `sklearn.feature_selection` kombinierst. Dieser Ansatz kann effektiver sein, um die optimale Feature-Set-Größe zu bestimmen, kann aber bei einer sehr großen Anzahl von Features zeitaufwendig sein.

### 3. **SHAP Values**

SHAP (SHapley Additive exPlanations) bietet eine fundierte und konsistente Methode, um die Beiträge jedes Features zu jeder einzelnen Vorhersage zu erklären. SHAP-Werte können dir helfen zu verstehen, wie dein Modell Entscheidungen trifft, und gleichzeitig die wichtigsten Features identifizieren. Die Nutzung von SHAP mit XGBoost kann aufschlussreich sein, erfordert jedoch zusätzliche Berechnungen.

### 4. **Feature Selection Using SelectFromModel**

`SelectFromModel` ist ein Meta-Transformer von scikit-learn, der Features nach ihrem Gewicht selektiert. Du kannst `SelectFromModel` mit XGBoost verwenden, um Features mit geringer Wichtigkeit automatisch zu entfernen. Diese Methode ermöglicht es dir, ein Schwellenwert für die Feature-Wichtigkeit festzulegen und kann eine effiziente Art der Feature-Reduktion sein.

### Beispielcode für Feature Importance mit XGBoost:

```python
from xgboost import XGBClassifier
from sklearn.feature_selection import SelectFromModel

# Angenommen, X_train und y_train sind bereits definiert
model = XGBClassifier()
model.fit(X_train, y_train)

# Auswahl der Features basierend auf Gewichtungsschwelle
thresholds = SelectFromModel(model, prefit=True)
X_train_reduced = thresholds.transform(X_train)

# X_train_reduced enthält nun nur noch die Features, die die Schwelle überschreiten
```

Jede dieser Methoden hat ihre eigenen Stärken und Einschränkungen. Es könnte sinnvoll sein, mit einigen davon zu experimentieren, um zu sehen, welche Methode die besten Ergebnisse für dein spezifisches Problem liefert. Beachte, dass die Effektivität der Feature-Auswahl stark von deinen Daten und dem spezifischen Problem, das du lösen möchtest, abhängt.